### EDA

In [ ]:
library(cowplot) #Installing package into 'C:/Users/User/Documents/R/win-library/4.0'
library(grid)
library(ggplot2)
library(ggExtra)
library(tidyverse)
library(lubridate)
library(gridExtra)
library(pROC)
library(ROCR)
library(ROCit)
library(caret)
library(boot)
library(gbm)
# library("cvAUC")

library(MLmetrics)

In [ ]:
# directories
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"
modeldir4 = "../../OutputTD/3_models/1_4_cohort"
# modeldir4preadmit = "../../OutputTD/3_models/1_4_cohort_24hrpreadmit"
tabledir = "../../OutputTD/4_tables"
resultdir = "../../OutputTD/5_results_analysis"

options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

In [ ]:
cohort <- read.csv(file.path(cohortdir, "1_4_cohort.csv")) %>% mutate(admit_time = ymd_hms(admit_time))
nrow(cohort)

demo <- read.csv(file.path(featuredir, "2_1_coh2_demo.csv"))  %>% mutate(admit_time = ymd_hms(admit_time)) %>% 
            select(anon_id, pat_enc_csn_id_coded, race)
nrow(demo)

esi <- read.csv(file.path(featuredir, "2_5_coh3_imputedHWESI.csv"))  %>% mutate(admit_time = ymd_hms(admit_time)) %>%
            select(anon_id, pat_enc_csn_id_coded, ESI_i) %>% rename(ESI = ESI_i)
nrow(esi)

los <- read.csv(file.path(datadir, "length_of_stay_labels.csv")) %>% mutate(admit_time = ymd_hms(admit_time))
nrow(los) # less than cohort

# adm_year = year(admit_time)

In [ ]:
features <- read.csv(file.path(featuredir, "2_7_coh4_feature_values.csv"))
nrow(features)
colnames(features)

In [17]:
unique(features$features)

[1] "ESI_i"                 "delta_ESI"             "gender"               
 [4] "age"                   "insurance"             "English"              
 [7] "Height_i"              "delta_H"               "Weight_i"             
[10] "delta_W"               "race.Asian"            "race.Black"           
[13] "race.Native.American"  "race.Other"            "race.Pacific.Islander"
[16] "race.Unknown"          "race.White"            "Temp"                 
[19] "DBP"                   "SBP"                   "Pulse"                
[22] "RR"                    "Lactate"               "PO2_a"                
[25] "pH_a"                  "TCO2_a"                "AnionGap"             
[28] "HCO3_a"                "O2sat_a"               "pCO2_a"               
[31] "Neut"                  "HCO3_v"                "BUN"                  
[34] "Cl"                    "K"                     "Base"                 
[37] "Ca"                    "Cr"                    "Na"                   
[40] "PT"                    "TProtein"              "ALB"                  
[43] "ALT"                   "AST"                   "CO2"                  
[46] "Glucose"               "Hct"                   "Hgb"                  
[49] "INR"                   "MCH"                   "pH_v"                 
[52] "Platelet"              "RDW"                   "Trop"                 
[55] "WBC"                   "ALK"                   "eGFR"                 
[58] "Glob"                  "PO2_v"                 "TBili"                
[61] "Eos"                   "Lymp"                  "pCO2_v"               
[64] "Basos"                 "Mono"                  "O2sat_v"

In [ ]:
head(cohort,1)
head(demo,1)
head(esi)
head(los)

### check ESI by race

In [ ]:
coh_demo <- left_join(cohort, demo) %>% left_join(esi) %>% mutate(race=factor(race))
nrow(coh_demo)
summary(coh_demo)

hist(coh_demo$ESI)

In [ ]:
race_esi <- coh_demo %>% group_by(race, ESI) %>% summarise(n = n()) %>% 
                group_by(race) %>% mutate(ntotal = sum(n), group_percentage = round(100*n/ntotal, 2))
race_esi

In [ ]:
color_table <- tibble(
  race = c("Asian", "Black", "Native American", "Other", "Pacific Islander", "Unknown", "White"),
  color = c("tan", "grey30", "salmon4", "magenta3", "tan3", "limegreen", "moccasin")
  )
color_table5 <- tibble(
  race = c("Asian", "Black", "Other", "White"),
  color = c("tan", "grey30", "magenta3", "moccasin")
  )

race_esi1 <- race_esi %>% filter(ESI==1)
race_esi23 <- race_esi %>% filter(ESI==2 | ESI==3)
race_esi4 <- race_esi %>% filter(ESI==4)
race_esi5 <- race_esi %>% filter(ESI==5)

race_esi5

In [ ]:
options(repr.plot.width=15, repr.plot.height=7)

p1 <- ggplot(race_esi1, aes(fill=race, y=group_percentage, x=race)) + 
    geom_bar(position="dodge", stat="identity") + scale_fill_manual(values = color_table$color) +
    geom_text(aes(label=n), position=position_dodge(width=1), vjust=-0.5)

p23 <- ggplot(race_esi23, aes(fill=race, y=group_percentage, x=ESI)) + 
    geom_bar(position="dodge", stat="identity") + scale_fill_manual(values = color_table$color) #+
#     geom_text(aes(label=n), position=position_dodge(width=1), vjust=0, angle=90)

grid.arrange(p1, p23, ncol=2)
ggsave(file.path(resultdir,"Fig_ESI123.png"), width = 15, height = 7, dpi = 1200) 

In [ ]:
p4 <- ggplot(race_esi4, aes(fill=race, y=group_percentage, x=race)) + 
    geom_bar(position="dodge", stat="identity") + scale_fill_manual(values = color_table$color) +
    geom_text(aes(label=n), position=position_dodge(width=1), vjust=-0.5)

p5 <- ggplot(race_esi5, aes(fill=race, y=group_percentage, x=race)) + 
    geom_bar(position="dodge", stat="identity") + scale_fill_manual(values = color_table5$color) +
    geom_text(aes(label=n), position=position_dodge(width=1), vjust=-0.5)

grid.arrange(p4, p5, ncol=2)
# plotlist <- list(p4, p5)
# plot_grid(plotlist=plotlist)
ggsave(file.path(resultdir,"Fig_ESI45.png"), width = 15, height = 7, dpi = 1200) 

### density plot of predicted probabilities for test cohort


In [ ]:
coh_test <- read.csv(file.path(modeldir4, "1_4_cohort_test_results.csv")) %>% 
                select(pat_enc_csn_id_coded, first_label, death_24hr_max_label, death_24hr_recent_label, 
                       pred_first, pred_death_24hr_max, pred_death_24hr_recent, abs_diff0_24, transfer)
nrow(coh_test)
colnames(coh_test)
# preds_max <- coh_test %>% select(outcome = death_24hr_max_label, prediction = pred_death_24hr_max)
# preds_24hr <- coh_test %>% select(outcome = death_24hr_recent_label, prediction = pred_death_24hr_recent)

In [ ]:
### Plot function, using data0
plotfxn <- function(X, Xlab, Fill, data){
#     p0 <- data %>% ggplot(aes(x=X)) + geom_density() + facet_wrap(~race_recoded) # unable to use Fill here
    p1 <- data %>% ggplot(aes(x=X, fill=Fill)) +
        geom_density(alpha=0.3, position = 'identity') + # color="#e9ecef", 
        labs(fill="") + xlab(Xlab) 

    p2 <- p1 + theme(axis.text=element_text(size=14),
             axis.title=element_text(size=16), legend.key.size = unit(1, "cm"))#, face="bold"
#     return(list(p0, p2))
    return(p2)
}

In [ ]:
options(repr.plot.width=14, repr.plot.height=7)

# 1st and 2nd outcome (highest LOC or at 24th)
pmax <- plotfxn(X=coh_test$pred_death_24hr_max, Xlab="Predicted Probability", Fill=factor(coh_test$death_24hr_max_label), coh_test)
p24 <- plotfxn(X=coh_test$pred_death_24hr_recent, Xlab="Predicted Probability", Fill=factor(coh_test$death_24hr_recent_label), coh_test)

grid.arrange(pmax, p24, ncol=2)
# ggsave(file.path(resultdir,"Fig_pred_dens.png"), width = 14, height = 7, dpi = 1200) 

In [ ]:
# 1st and 2nd outcome (highest LOC or at 24th), with labels, combined legends

prow <- plot_grid(
            plotfxn(X=coh_test$pred_death_24hr_max, Xlab="Predicted Probability", 
                    Fill=factor(coh_test$death_24hr_max_label), coh_test) +
                theme(legend.position="none"), 
            plotfxn(X=coh_test$pred_death_24hr_recent, Xlab="Predicted Probability", 
                    Fill=factor(coh_test$death_24hr_recent_label), coh_test) +
                theme(legend.position="none"),
            labels = c("A: highest LOC", "B: time24 LOC"),
            hjust = -1,
            nrow = 1
)

legend <- get_legend(
            plotfxn(X=coh_test$pred_death_24hr_max, Xlab="Predicted Probability", 
                    Fill=factor(coh_test$death_24hr_max_label), coh_test) +
            theme(legend.title = element_text(color = "Black", size = 16),
                  legend.text = element_text(color = "black", size = 14)
                 ) + scale_fill_discrete(name = "LOC")#+ theme(legend.box.margin = margin(0, 0, 0, 1))
)

plot_grid(prow, legend, rel_widths=c(6, 0.7))
ggsave(file.path(resultdir,"Fig_pred_dens.png"), width = 14, height = 7, dpi = 1200) 

In [ ]:
# convert wide to long, entire test cohort
coh_test2 <- coh_test %>% rename(time0=pred_first, time24=pred_death_24hr_recent) %>% 
                select(-pred_death_24hr_max, -death_24hr_max_label)
coh_test2 <- gather(coh_test2, time, prediction, time0:time24, factor_key=TRUE)
head(coh_test2)

In [ ]:
# hist(coh_test$abs_diff0_24)
# plotfxn(X=cohdis$pred_death_24hr_recent, Xlab="Predicted Probability", Fill=factor(coh_test$death_24hr_recent_label), coh_test)

options(repr.plot.width=10, repr.plot.height=7)

# test cohort, with abs difference btwn predicted values at time0 and time24 greater than 0.1
ggplot(data = coh_test2, aes(x=prediction, fill=time)) +
        geom_density(alpha=0.3, position = 'identity') + # color="#e9ecef", 
        labs(fill="")

ggplot(data = coh_test[coh_test$abs_diff0_24 > 0.1, ], aes(x=abs_diff0_24)) +
        geom_histogram() + xlab("absolute difference in model predictions")

In [ ]:
# get discordance cohort abs_diff0_@4 >= 0.4
colnames(coh_test)
coh_dis <- coh_test %>% filter(abs_diff0_24 >= 0.4)
nrow(coh_dis)

In [ ]:
# coh_dis[coh_dis$transfer==1, ]$pred_first
# coh_dis[coh_dis$transfer==1 & coh_dis$first_label==1, ]$pred_first

In [ ]:
# discordance cohort, only with pred at time 24, not the primary outcome, wide to long
cohdis <- coh_dis %>% rename(time0=pred_first, time24=pred_death_24hr_recent) %>% 
            select(-pred_death_24hr_max, -death_24hr_max_label)

# get an index for each of the observations 
cohdis$idx <- rownames(cohdis)
nrow(cohdis)
head(cohdis)

# wide to long
cohdis2 <- gather(cohdis, time, prediction, time0:time24, factor_key=TRUE)
nrow(cohdis2)
head(cohdis2)

In [ ]:
# all discordance >= 0.4 in pairs, not very informative
options(repr.plot.width=20, repr.plot.height=7)

p <- ggplot(cohdis2, aes(idx, prediction)) + 
        geom_point(aes(colour = factor(time), shape = factor(time))) + 
        geom_line(aes(group = idx)) +
        theme(axis.title.x=element_blank(),
              axis.text.x=element_blank(),
              axis.ticks.x=element_blank())
p

In [ ]:
# density plot for prediction at time 0 and 24 for the discordance cohort
# time0 prob lean toward higher values and time24 leans toward lower values
plotfxn(X=cohdis2$prediction, Xlab="Predicted Probability", Fill=factor(cohdis2$time), cohdis2)

In [ ]:
# coh_dis[coh_dis$transfer==1, ]$pred_first
# coh_dis[coh_dis$transfer==1 & coh_dis$first_label==1, ]$pred_first

# discordance cohort who did transfer
dtx10 = cohdis2[cohdis2$transfer==1 & cohdis2$first_label==1, ]
dtx01 = cohdis2[cohdis2$transfer==1 & cohdis2$first_label==0, ]

pdtx10 <- plotfxn(X=dtx10$prediction, Xlab="Predicted Probability", Fill=factor(dtx10$time), dtx10)
pdtx01 <- plotfxn(X=dtx01$prediction, Xlab="Predicted Probability", Fill=factor(dtx01$time), dtx01)

options(repr.plot.width=15, repr.plot.height=7)
grid.arrange(pdtx10, pdtx01, ncol=2)

# ggsave(file.path(resultdir,"Fig_disc_tx_dens.png"), width = 14, height = 7, dpi = 1200) 

In [ ]:
# remove duplicate legends

prowt <- plot_grid(pdtx10 + theme(legend.position="none"), pdtx01 + theme(legend.position="none"),
#             labels = c("A: highest LOC", "B: time24 LOC"),
            hjust = -1,
            nrow = 1
)

# legend <- get_legend(
#             plotfxn(X=coh_test$pred_death_24hr_max, Xlab="Predicted Probability", 
#                     Fill=factor(coh_test$death_24hr_max_label), coh_test) +
#             theme(legend.title = element_text(color = "Black", size = 16),
#                   legend.text = element_text(color = "black", size = 14)
#                  ) + scale_fill_discrete(name = "LOC")#+ theme(legend.box.margin = margin(0, 0, 0, 1))
# )
plot_grid(prowt, legend, rel_widths=c(6, 0.7))

In [ ]:
# discordance cohort who did not transfer
# remove duplicate legends

d1 = cohdis2[cohdis2$transfer==0 & cohdis2$first_label==1, ]
d0 = cohdis2[cohdis2$transfer==0 & cohdis2$first_label==0, ]

pd1 <- plotfxn(X=d1$prediction, Xlab="Predicted Probability", Fill=factor(d1$time), d1)
pd0 <- plotfxn(X=d0$prediction, Xlab="Predicted Probability", Fill=factor(d0$time), d0)

# plot
options(repr.plot.width=15, repr.plot.height=7)
# grid.arrange(pd1, pd0, ncol=2)

prowd <- plot_grid(pd1 + theme(legend.position="none"), pd0 + theme(legend.position="none"),
#             labels = c("A: highest LOC", "B: time24 LOC"),
            hjust = -1,
            nrow = 1
)

plot_grid(prowd, legend, rel_widths=c(6, 0.7))
# ggsave(file.path(resultdir,"Fig_disc_notx_dens.png"), width = 14, height = 7, dpi = 1200) 

### check LOS and death
This has less than the full cohort

In [ ]:
los <- read_csv(file.path(resultdir, "5_2_length_of_stay_labels.csv"))
nrow(los)

In [ ]:
head(los, n=1)

In [ ]:
table(los$died_before_discharge)
table(los$length_of_ip_since_admit)
table(los$length_from_ED_entry_until_end_date)

In [ ]:
los <- los %>% mutate(died_before_discharge = ifelse(died_before_discharge, 1, 0),
                      death_dc_days = death_date_jittered - discharge_date,
                      admit_ed_hours = round(as.numeric(difftime(admit_time, first_ED_time, units="hours")), 1),
#                       length_of_ip_since_admit = str_sub(length_of_ip_since_admit, star=1, end=-5), 
#                       length_from_ED_entry_until_end_date = str_sub(length_from_ED_entry_until_end_date, star=1, end=-5))
                      length_of_ip_since_admit= ifelse(length_of_ip_since_admit == "", NA,
                                                       as.numeric(str_sub(length_of_ip_since_admit, star=1, end=-5))),
                      length_from_ED_entry_until_end_date= ifelse(length_from_ED_entry_until_end_date == "", NA,
                                                                  as.numeric(str_sub(length_from_ED_entry_until_end_date, star=1, end=-5))))
head(los, n=1)

In [ ]:
los <- los %>% mutate(death_dc_days = as.numeric(death_dc_days))
head(los)

In [ ]:
table(los$died_before_discharge)
summary(los$length_of_ip_since_admit)
summary(los$length_from_ED_entry_until_end_date)
summary(los[los$died_before_discharge == 1, ]$death_dc_days)

In [ ]:
los %>% filter(death_dc_days < -1)
los %>% filter(length_of_ip_since_admit < 0)
los %>% filter(length_from_ED_entry_until_end_date < 0)

In [ ]:
hist(los$length_of_ip_since_admit)
hist(los$length_from_ED_entry_until_end_date)

### read the discordance

In [ ]:
los_dis <- read_csv(file.path(tabledir, "4_2_diffIDs_117.csv")) 
nrow(los_dis)

In [ ]:
los_tx <- cohort %>% select(pat_enc_csn_id_coded, first_label, death_24hr_recent_label) %>% 
            filter(first_label != death_24hr_recent_label) %>% left_join(los)
nrow(los_tx)

In [ ]:
los_dis <- right_join(los, los_dis)
nrow(los_dis)
head(los_dis)

In [ ]:
table(los$died_before_discharge)
table(los_dis$died_before_discharge)
table(los_tx$died_before_discharge)

summary(los$length_of_ip_since_admit)
summary(los_dis$length_of_ip_since_admit)
summary(los_tx$length_of_ip_since_admit)

summary(los$admit_ed_hours)
summary(los_dis$admit_ed_hours)
summary(los_tx$admit_ed_hours)

In [ ]:
los_tx10 <- los_tx %>% filter(first_label==1)
los_tx01 <- los_tx %>% filter(first_label==0)

nrow(los_tx01)
nrow(los_tx10)

table(los_tx10$died_before_discharge)
table(los_tx01$died_before_discharge)

summary(los_tx$length_of_ip_since_admit)
summary(los_tx01$length_of_ip_since_admit)

summary(los_tx$admit_ed_hours)
summary(los_tx01$admit_ed_hours)

In [ ]:
txdeath <- los_tx %>% filter(died_before_discharge ==1) %>% select(anon_id)
write.csv(txdeath, file.path(resultdir, "5_3_tx_death.csv"), row.names=FALSE)
nrow(txdeath)